In [3]:
import subprocess
import os
import zipfile
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureCollection, FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.geometry import Geometry, Point, Polyline, lengths
import csv
from collections import defaultdict

In [4]:
# Method 2: Logging in with a client ID
gis_url= "https://amherstcollege.maps.arcgis.com" # url to your org's arcGIS
your_client_id = "Z1lqH7hW52EWwbxZ"               # paste client ID here
gis = GIS(gis_url, client_id=your_client_id)
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://amherstcollege.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Z1lqH7hW52EWwbxZ&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=iUm8UpmM67oNxCbVpQAuhuqVfkS0e4&allow_verification=false


Enter code obtained on signing in using SAML:  ········


Successfully logged in as: tansar27_amherstcollege


In [5]:
buffers = gis.content.search(
    f"title:Idaho,Buffer,leq AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(where="1=1", out_fields="*", return_geometry=True).features
    bf_features = []
    counts = []
    building_lists = []  # Store actual building features per geometry

    for buffer_feature in buffer_features:
        buffer_geom = buffer_feature.geometry
        buildings_clipped = buildings.query(
            geometry_filter=intersects(buffer_geom),
            return_geometry=True
        )
        count = len(buildings_clipped.features)
        counts.append(count)
        building_lists.append(buildings_clipped.features)  # keep full features
        bf_features.append(buffer_feature.attributes)  # store here
        
    site_index = 1
    for i in range(0, len(counts), 2):
        count_x = counts[i]
        buildings_x = building_lists[i]
        #tot_score_x = tot_scores[i]  # retrieve stored tot_score
        bf_features_x = bf_features[i] 

        if i + 1 < len(counts):
            count_y = counts[i + 1]
            buildings_y = building_lists[i + 1]
        else:
            count_y = 0
            buildings_y = []

        total_count = count_x + count_y
        #site_name = f"{site_index}_{tot_score_x}_{buffer.title[:-12]}"
        print("==========")
        print(bf_features_x)
        print("==========")
        #print(f"{site_name}: {total_count} building(s)")
        if total_count > 0:
            print("Longitude and latitude for each building:")
            for b in buildings_x + buildings_y:
                long = b.attributes.get("LONGITUDE")
                lat = b.attributes.get("LATITUDE")
                oid = b.attributes.get("BUILD_ID")
               
        else:
            print("No buildings found")
        print("----------")
        site_index += 1

        

{'OBJECTID': 1, 'id': ' ', 'Name': 'x arm', 'descriptio': ' ', 'timestamp': ' ', 'begin_': ' ', 'end_': ' ', 'altitudeMo': ' ', 'tessellate': -1, 'extrude': 0, 'visibility': -1, 'drawOrder': 0, 'icon': ' ', 'arm_length': 40000, 'open_angle': 90, 'rot_angle': 220, 'vert_x': -1326736.65243242, 'vert_y': 2443783.07180378, 'elev_score': 32.40826925, 'tilt_score': 0.56794002, 'lc_score': 19.74, 'tot_score': 52.71620928, 'BUFF_DIST': 200, 'ORIG_FID': 1, 'AnalysisArea': 15.98868229}
Longitude and latitude for each building:
----------
{'OBJECTID': 3, 'id': ' ', 'Name': 'x arm', 'descriptio': ' ', 'timestamp': ' ', 'begin_': ' ', 'end_': ' ', 'altitudeMo': ' ', 'tessellate': -1, 'extrude': 0, 'visibility': -1, 'drawOrder': 0, 'icon': ' ', 'arm_length': 40000, 'open_angle': 90, 'rot_angle': 230, 'vert_x': -1337629.29752454, 'vert_y': 2434870.9076375, 'elev_score': 33.45683881, 'tilt_score': 8.47350773, 'lc_score': 5.94, 'tot_score': 47.87034654, 'BUFF_DIST': 200, 'ORIG_FID': 3, 'AnalysisArea': 

In [10]:
buffers = gis.content.search(
    f"title:Idaho,Buffer, leq AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

buffer_feature_groups = defaultdict(list)
zero_building_pairs = []

output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts1.csv"
output_csv_detailed = r"C:\Users\user\Downloads\CE_sites_data\Idaho_counts_coords1.csv"

# STEP 1: Group buffer features by (vert_x, vert_y, tot_score)
for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(
        where="1=1", out_fields="*", return_geometry=True
    ).features

    for feature in buffer_features:
        attrs = feature.attributes
        key = (attrs["vert_x"], attrs["vert_y"], attrs["tot_score"])
        buffer_feature_groups[key].append(feature)
        feature.attributes["buffer_title"] = buffer.title
        
site_index = 1
# STEP 2: Process each buffer group ONCE
for key, feature_list in buffer_feature_groups.items():
    if len(feature_list) != 2:
        print(f"More than a pair of arms ({len(feature_list)}) detected for vert_x, vert_y, tot_score: {key}")
        continue

    total_count = 0
    all_buildings = []

    for feature in feature_list:
        buffer_geom = feature.geometry
        buildings_clipped = buildings.query(
            geometry_filter=intersects(buffer_geom),
            return_geometry=True
        )
        count = len(buildings_clipped.features)
        total_count += count
        all_buildings.extend(buildings_clipped.features)

    buffer_title = feature_list[0].attributes["buffer_title"]
    site_name = f"{site_index}_{int(key[2])}_{buffer_title[:-7]}"
    site_index += 1
    
    if total_count == 0:
        print(f"{site_name}: 0")
        zero_building_pairs.append({
            "vert_x": key[0],
            "vert_y": key[1],
            "tot_score": key[2]
        })
       # results.append([site_name, total_count, None, None, None])
    else:
        #for b in all_buildings:
           # results.append([
            #    site_name,
              #  total_count,
               # b.attributes.get("BUILD_ID"),
                #b.attributes.get("LONGITUDE"),
                #b.attributes.get("LATITUDE")
            #])
        print(f"{site_name}: {total_count} building(s)")



1_52_Idaho_Falls_leq_60_200m: 7 building(s)
2_47_Idaho_Falls_leq_60_200m: 0
3_59_Idaho_Falls_leq_60_200m: 1 building(s)
4_53_Idaho_Falls_leq_60_200m: 0
5_54_Idaho_Falls_leq_60_200m: 0
6_57_Idaho_Falls_leq_60_200m: 42 building(s)
7_59_Idaho_Falls_leq_60_200m: 11 building(s)


In [13]:
buffers = gis.content.search(
    f"title:Idaho,Buffer AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)

buildings_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/USA_Structures_View/FeatureServer/0"
buildings = FeatureLayer(buildings_url)

# Group buffer features by title, then (vert_x, vert_y, tot_score)
buffer_title_groups = defaultdict(lambda: defaultdict(list))
summary = []
results = []
zero_buildings = []
zero_building_pairs = []

# Step 1: Organize features into nested dict
for buffer in buffers:
    input_buffer = buffer.layers[0]
    buffer_features = input_buffer.query(
        where="1=1", out_fields="*", return_geometry=True
    ).features

    for feature in buffer_features:
        attrs = feature.attributes
        buffer_title = buffer.title
        key = (attrs["vert_x"], attrs["vert_y"], attrs["tot_score"])
        feature.attributes["buffer_title"] = buffer_title  
        buffer_title_groups[buffer_title][key].append(feature)

for buffer_title, feature_groups in buffer_title_groups.items():
    site_index = 1 

    for key, feature_list in feature_groups.items():
        if len(feature_list) != 2:
            print(f"More than a pair of arms ({len(feature_list)}) detected for key {key} in buffer '{buffer_title}'")
            continue

        total_count = 0
        all_buildings = []

        for feature in feature_list:
            buffer_geom = feature.geometry
            buildings_clipped = buildings.query(
                geometry_filter=intersects(buffer_geom),
                return_geometry=True
            )
            count = len(buildings_clipped.features)
            total_count += count
            all_buildings.extend(buildings_clipped.features)

        site_name = f"{site_index}_{int(key[2])}_{buffer_title[:-7]}"
        site_index += 1
        summary.append([site_name, total_count])
        if total_count == 0:
            print(f"{site_name}: 0")
            zero_building_pairs.append({
                "vert_x": key[0],
                "vert_y": key[1],
                "tot_score": key[2],
            })
            results.append([site_name, total_count, None, None, None])
            zero_buildings.append([site_name, total_count])
        else:
            for b in all_buildings:
                results.append([site_name,total_count,b.attributes.get("BUILD_ID"),b.attributes.get("LONGITUDE"),b.attributes.get("LATITUDE")])
            print(f"{site_name}: {total_count} building(s)")

output_summary_name = "Idaho_building_counts_200m.csv"
output_detailed_name = "Idaho_building_coordinates_200m.csv"
output_zero_name = "Idaho_no_buildings_200m.csv"

output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts_200m.csv"
output_csv_detailed = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_coordinates_200m.csv"
output_csv_zero = r"C:\Users\user\Downloads\CE_sites_data\Idaho_no_buildings_200m.csv"

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


with open(output_csv_detailed, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings","Building ID", "Longitude", "Latitude"])
    writer.writerows(results)

print(f"CSV saved to {os.path.abspath(output_csv_detailed)}")

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


1_79_Idaho_Falls_60to80_200m: 17 building(s)
2_74_Idaho_Falls_60to80_200m: 0
3_79_Idaho_Falls_60to80_200m: 3 building(s)
4_75_Idaho_Falls_60to80_200m: 0
5_76_Idaho_Falls_60to80_200m: 11 building(s)
6_79_Idaho_Falls_60to80_200m: 0
7_79_Idaho_Falls_60to80_200m: 13 building(s)
8_63_Idaho_Falls_60to80_200m: 14 building(s)
9_63_Idaho_Falls_60to80_200m: 2 building(s)
10_68_Idaho_Falls_60to80_200m: 0
11_71_Idaho_Falls_60to80_200m: 0
12_78_Idaho_Falls_60to80_200m: 0
13_70_Idaho_Falls_60to80_200m: 1 building(s)
14_69_Idaho_Falls_60to80_200m: 33 building(s)
15_61_Idaho_Falls_60to80_200m: 0
16_63_Idaho_Falls_60to80_200m: 7 building(s)
17_69_Idaho_Falls_60to80_200m: 7 building(s)
18_61_Idaho_Falls_60to80_200m: 9 building(s)
19_60_Idaho_Falls_60to80_200m: 16 building(s)
20_64_Idaho_Falls_60to80_200m: 36 building(s)
21_77_Idaho_Falls_60to80_200m: 26 building(s)
22_65_Idaho_Falls_60to80_200m: 31 building(s)
23_74_Idaho_Falls_60to80_200m: 0
24_63_Idaho_Falls_60to80_200m: 0
25_72_Idaho_Falls_60to80_200

In [28]:
print(len(buffer_title_groups))
output_csv_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts_200m.csv"

with open(output_csv_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Site Name", "Number of Buildings"])
    writer.writerows(summary)

print(f"CSV saved to {os.path.abspath(output_csv_summary)}")


5
CSV saved to C:\Users\user\Downloads\CE_sites_data\Idaho_building_counts_200m.csv


In [27]:
from pykml import parser
from lxml import etree
import os

reference_dicts = zero_building_pairs
reference_set = {(d["vert_x"], d["vert_y"], d["tot_score"]) for d in reference_dicts}
print(reference_set)
input_folder = r"C:\Users\user\Downloads\IdahoFalls"
output_folder = r"C:\Users\user\Downloads\IdahoFallsZeroBuildings"
os.makedirs(output_folder, exist_ok=True)
kml_ns = "{http://www.opengis.net/kml/2.2}"

def matches_reference(vx, vy, ts, ref_set, tol=1e-6):
    for rvx, rvy, rts in ref_set:
        if abs(vx - rvx) < tol and abs(vy - rvy) < tol and abs(ts - rts) < tol:
            return True
    return False

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".kml"):
        kml_path = os.path.join(input_folder, filename)
        with open(kml_path) as f:
            doc = parser.parse(f).getroot()
           
            for placemark in list(doc.findall(f".//{kml_ns}Placemark")):
                try:
                    simple_data = placemark.findall(f".//{kml_ns}SimpleData")
                    data_dict = {sd.get('name'): sd.text for sd in simple_data}
                    vx = float(data_dict.get('vert_x', 'nan'))
                    vy = float(data_dict.get('vert_y', 'nan'))
                    ts = float(data_dict.get('tot_score', 'nan'))

                    if not matches_reference(vx, vy, ts, reference_set):
                        placemark.getparent().remove(placemark)
                except Exception:
                    # Skip placemarks with invalid or missing attributes
                    placemark.getparent().remove(placemark)
       
        modified_name = f"{filename[:-4]}" + "_zero_buildings_200m.kml"
        print(modified_name)
        output_path = os.path.join(output_folder, modified_name)
        with open(output_path, "wb") as f:
            f.write(etree.tostring(doc, pretty_print=True, xml_declaration=True, encoding="UTF-8"))
       
        print(f"Saved KML with no buildings to to {output_path}")



{(-1419819.25594695, 2340798.06366004, 117.28691336), (-1376248.67557845, 2370505.27754766, 132.48926335), (-1375258.43511553, 2369515.03708474, 128.51888716), (-1412887.57270651, 2356641.91106677, 125.18682118), (-1442594.78659413, 2341788.30412296, 125.56059293), (-1441604.54613121, 2362583.3538443, 131.36007723), (-1411897.33224359, 2345749.26597465, 113.31876257), (-1395063.24437394, 2344759.02551173, 99.70537313), (-1374268.19465261, 2364563.83477014, 119.744227), (-1355453.62585711, 2422988.02208245, 118.03675559), (-1359414.5877088, 2422988.02208245, 122.49906615), (-1457448.39353794, 2350700.46828925, 111.01624153), (-1397043.72529978, 2346739.50643757, 130.09940503), (-1397043.72529978, 2341788.30412296, 106.51031831), (-1373277.95418969, 2365554.07523306, 122.23837342), (-1445565.50798289, 2347729.74690049, 131.49993212), (-1420809.49640988, 2362583.3538443, 133.48055662), (-1347531.70215375, 2428929.46485997, 122.33584458), (-1343570.74030207, 2433880.66717457, 103.93892659)

In [55]:
import os
import csv
from pykml import parser
from lxml import etree

# Your reference data (no-building sites)
reference_dicts = zero_building_pairs
reference_set = {(d["vert_x"], d["vert_y"], d["tot_score"]) for d in reference_dicts}

# Namespace
kml_ns = "{http://www.opengis.net/kml/2.2}"

# Paths
input_folder = r"C:\Users\user\Downloads\IdahoFalls"
output_kml_summary = r"C:\Users\user\Downloads\CE_sites_data\Idaho_sorted_by_buildings_200m.csv"

# Matching function
def matches_reference(vx, vy, ts, ref_set, tol=1e-6):
    for rvx, rvy, rts in ref_set:
        if abs(vx - rvx) < tol and abs(vy - rvy) < tol and abs(ts - rts) < tol:
            return True
    return False

# Result list
results = []

# Loop through each input KML file
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".kml"):
        kml_path = os.path.join(input_folder, filename)

        with open(kml_path, 'r', encoding='utf-8') as f:
            doc = parser.parse(f).getroot()
            placemarks = list(doc.findall(f".//{kml_ns}Placemark"))
            total_sites = int(len(placemarks) / 2)

            matched_count = 0  # This will be sites_nb

            for placemark in placemarks:
                try:
                    simple_data = placemark.findall(f".//{kml_ns}SimpleData")
                    data_dict = {sd.get('name'): sd.text for sd in simple_data}
                    vx = float(data_dict.get('vert_x', 'nan'))
                    vy = float(data_dict.get('vert_y', 'nan'))
                    ts = float(data_dict.get('tot_score', 'nan'))

                    if matches_reference(vx, vy, ts, reference_set):
                        matched_count += 1
                except Exception:
                    continue  # Skip if any attribute is missing

            sites_nb = int(matched_count / 2)
           
            results.append([filename, total_sites, sites_nb])

# Write summary CSV
with open(output_kml_summary, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["KML Name", "Total sites", "Number of sites with no buildings"])
    writer.writerows(results)

print(f"Summary CSV saved to: {os.path.abspath(output_kml_summary)}")



Summary CSV saved to: C:\Users\user\Downloads\CE_sites_data\Idaho_sorted_by_buildings_200m.csv
